In [57]:
import sys        
import random
import warnings
import time
import numpy as np
import pandas as pd
import collections, functools, operator
import os
import pickle

# Parameters of system
warnings.filterwarnings("ignore")

In [58]:
os.chdir('/Users/moises/Desktop/Produccion-Tesis/')

replicas = 1

for experiment in [1, 2, 4, 5, 6]:            
    file_name = f"Resultados/Escenario {experiment}/"
    df = pd.read_pickle(file_name+"objective_value.pkl")
    print(f"Escenario {experiment}: {df['value'].sum()/replicas}")

Escenario 1: 187266979.79284772
Escenario 2: 187066208.72272825
Escenario 4: 187651217.07275575
Escenario 5: 188163505.75720635
Escenario 6: 183856704.7108145


In [ ]:
final = pd.DataFrame()
for experiment in range (1, 7):
    # Recorrer cada experimento
    if experiment in [1, 2,  3, 4, 5, 6]:
        file_name = f"Resultados/Escenario {experiment}/"
        df = pd.read_pickle(file_name+"objective_value.pkl").rename(columns={'value': 'utilidad'}).groupby(['r']).sum().reset_index()['utilidad']
        df1 = pd.read_pickle(file_name+"ingresos.pkl").rename(columns={'value': 'ingresos'}).groupby(['r']).sum().reset_index()['ingresos']
        df2 = pd.read_pickle(file_name+"costo_inventario.pkl").rename(columns={'value': 'costo_inventario'}).groupby(['r']).sum().reset_index()['costo_inventario']
        df3 = pd.read_pickle(file_name+"costo_corte.pkl").rename(columns={'value': 'costo_corte'}).groupby(['r']).sum().reset_index()['costo_corte']
        df4 = pd.read_pickle(file_name+"costo_merma.pkl").rename(columns={'value': 'costo_merma'}).groupby(['r']).sum().reset_index()['costo_merma']
        df = pd.concat([df, df1, df2, df3, df4], axis=1)
        df['Escenario'] = f"Escenario {experiment}"
        df = df.reset_index().rename(columns={'index': 'r'})
        df['r'] += 1

        # Se agrega al DataFrame 
        final = pd.concat([final, df])
        
        fila = f"Escenario {experiment} & "
        for metrica in ['utilidad', 'ingresos', 'costo_inventario', 'costo_corte', 'costo_merma']:
            # Calcula la media y desviación estándar de la columna 'utilidad'
            media = df[metrica].mean()
            desviacion_estandar = df[metrica].std()

            # Calcula el tamaño de la muestra
            tamano_muestra = len(df)

            # Define el nivel de confianza deseado (por ejemplo, 95%)
            nivel_confianza = 0.95

            # Calcula el valor crítico de la distribución t de Student
            valor_critico = stats.t.ppf((1 + nivel_confianza) / 2, tamano_muestra - 1)

            # Calcula el error estándar
            error_estandar = desviacion_estandar / np.sqrt(tamano_muestra)

            # Calcula el intervalo de confianza
            intervalo_inferior = media - valor_critico * error_estandar
            intervalo_superior = media + valor_critico * error_estandar

            # Expresa el intervalo de confianza en potencia de 10e6 (potencia de 6)
            media = "{:.4f}".format(media * 1e-6)
            error = "{:.4f}".format(valor_critico * error_estandar * 1e-6)

            # Imprime el intervalo de confianza expresado en potencia de 10e6 (potencia de 6)
            cadena = f"${media}$ $\pm$ ${error}$ $\\times10^-6_$".replace('-', '{'). replace('_', '}')
            #print(f"{metrica}:{cadena}")

            fila += f"{cadena} & "

        fila =  fila[:-3]
        if experiment != 6:
            fila += "\\\\ \\midrule "
        else:
            fila += "\\\\ \\bottomrule "

        print(fila)

        
#final
final.to_excel("/Users/moises/Desktop/Codigo - Test significancia/ANOVA.xlsx", sheet_name="ANOVA")    


In [ ]:
import json
with open('random.json', 'r') as fp:
    ERROR_DDA = json.load(fp)

import pandas as pd

def flatten_dict(d, parent_key='', sep='_'):
    flattened_dict = {}
    for k, v in d.items():
        new_key = parent_key + sep + k if parent_key else k
        if isinstance(v, dict):
            flattened_dict.update(flatten_dict(v, new_key, sep))
        else:
            flattened_dict[new_key] = v
    return flattened_dict

# Ejemplo de diccionario
diccionario = ERROR_DDA

# Aplicar la función flatten_dict para aplanar el diccionario
flattened_dict = flatten_dict(diccionario)

# Convertir el diccionario a un dataframe
df = pd.DataFrame.from_dict(flattened_dict, orient='index', columns=['value']).to_excel('random.xlsx')

